### Standford Cars Vehicle Recognition - CNN modeling Xception ROUGH DRAFT

### CODE ONLY, For a detailed report, please refer to the Final Report

#### by Sean Sungil Kim

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
# importing necessary modules
import numpy as np
import pandas as pd
import seaborn as sns
import cv2
import keras
import keras.backend as K
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Model
#from keras.applications.resnet50 import ResNet50
#from keras.applications.inception_v3 import InceptionV3
from keras.applications import Xception
import tensorflow as tf
#import fastai
import matplotlib.pyplot as plt
from sklearn.utils.class_weight import compute_class_weight
from sklearn.model_selection import train_test_split
from sklearn.metrics import confusion_matrix, classification_report, precision_recall_fscore_support
import time
import warnings
warnings.filterwarnings("ignore")

# custom python scripts
import SC_sungil             # preprocessing
import ConvNet_sungil        # ConvNet modeling

In [ ]:
# for convenience, vehicle detected images are already saved
# loading the pre-saved vehicle detected images, classes and labels
train_detected_sc, train_data_class, data_labels = SC_sungil.load_images('saved_images/resized/training/',\
                                            'devkit/cars_train_annos.mat', None, 'devkit/cars_meta.mat')
test_detected_sc, test_data_class = SC_sungil.load_images('saved_images/resized/testing/', None,\
                                                          'devkit/cars_test_annos_withlabels.mat', None)

In [ ]:
# combining the training and testing dataset
all_data, all_class = SC_sungil.comb_data(train_detected_sc, test_detected_sc,\
                                          train_data_class, test_data_class)

# removing year and mergining classes
new_data_labels, all_class = SC_sungil.rmv_year(data_labels, all_class)
all_class = all_class - 1

# number of classes
num_classes = len(new_data_labels)

In [ ]:
# 80% train 20% test split
# test_size was set to 0.1226, to maintain the validation set size to 20% post under-sampling
x_train, x_test, y_train, y_test = train_test_split(all_data, all_class, stratify = all_class,\
                                                    test_size = 0.1226)

# random under-sampling
und_x_train, und_y_train = SC_sungil.under_sample(x_train, y_train)

# converting a class vector to binary class matrix
y_train_bin = keras.utils.to_categorical(und_y_train, num_classes)
y_test_bin = keras.utils.to_categorical(y_test, num_classes)

# post split class distribution
fig = plt.figure(figsize = (15, 5))
plt.subplot(1, 3, 1), sns.distplot(all_class, bins = num_classes)
plt.title('Distribution Plot of Classes in the Original Data', y = 1.02)
plt.xlabel('Class Number'), plt.ylabel('Density')
plt.subplot(1, 3, 2), sns.distplot(und_y_train, bins = num_classes)
plt.title('Distribution Plot of Classes in the Re-split Training Set\n(Under-sampled)', y = 1.02)
plt.xlabel('Class Number'), plt.ylabel('Density')
plt.subplot(1, 3, 3), sns.distplot(y_test, bins = num_classes)
plt.title('Distribution Plot of Classes in the Re-split Testing Set', y = 1.02)
plt.xlabel('Class Number'), plt.ylabel('Density')
plt.show()

tot_len = len(und_x_train) + len(x_test)
print('Total of %i images in the training data' % len(und_x_train))
print('Total of %i images in the testing data' % len(x_test))
print('%0.2f percent training set, %0.2f percent testing set' \
      % (len(und_x_train)/tot_len*100, len(x_test)/tot_len*100))

### Exploring with State-of-the-Art CNN Architectures

#### Xception

In [ ]:
# compute quantities required for featurewise normalization
# (std, mean, and principal components if ZCA whitening is applied)
train_datagen = ImageDataGenerator(rescale = 1. / 255, rotation_range = 40, width_shift_range = 0.2,\
                    height_shift_range = 0.2, shear_range = 0.2, zoom_range = 0.2, horizontal_flip = True,\
                    fill_mode = 'nearest')
test_datagen = ImageDataGenerator(rescale = 1. / 255)

train_datagen.fit(und_x_train)
test_datagen.fit(x_test)

In [ ]:
# base pre-trained InceptionV3 model
#base_model = Xception(include_top = False, weights = 'imagenet', input_shape = und_x_train.shape[1:4])

# global spatial average pooling, flattening, fully-connected, dropout and logistic layer
#x = base_model.output
#x = keras.layers.GlobalAveragePooling2D()(x)
#x = keras.layers.Dense(2048, activation = 'relu')(x)
#x = keras.layers.Dropout(0.4)(x)
#predictions = keras.layers.Dense(num_classes, activation = 'softmax')(x)

# model1
#model1 = Model(inputs = base_model.input, outputs = predictions)
#model1.summary()

In [ ]:
# freezing all convolutional InceptionV3 layers to train only the top layers which were randomly initialized
#for layer in base_model.layers:
#    layer.trainable = False

# compiling the model post freezing
#model1.compile(optimizer = keras.optimizers.Adam(lr = 0.0001), loss = 'categorical_crossentropy',\
#               metrics = ['accuracy'])

# learning rate finder
#start_ts = time.time()
#lr_finder1 = ConvNet_sungil.lr_finder(model1)
#lr_finder1.find_generator(train_datagen.flow(und_x_train, y_train_bin, batch_size = 256),\
#                          start_lr = 0.00001, end_lr = 0.1, epochs = 2, steps_per_epoch = len(und_x_train) / 256)
#print("Total Runtime:", time.time() - start_ts)

In [ ]:
#lr_finder1.plot_loss(n_skip_beginning = 1)
#lr_finder1.plot_loss_change(n_skip_beginning = 1, sma = 20)

In [ ]:
# setting the optimied learning rate
#K.set_value(model1.optimizer.lr, 0.0001)

# model checkpoint
#mc = keras.callbacks.ModelCheckpoint('best_model1.h5', monitor = 'val_acc', mode = 'max',\
#                                     verbose = 1, save_best_only = True)

# fitting the model on batches with real-time data augmentation
# training the model (top layers) on the new data for 10 epochs
#start_ts = time.time()
#history1 = model1.fit_generator(train_datagen.flow(und_x_train, y_train_bin, batch_size = 256),
#    steps_per_epoch = len(und_x_train) / 256, epochs = 10, callbacks = [mc],\
#    validation_data = test_datagen.flow(x_test, y_test_bin), validation_steps = len(x_test) / 256)
#print("Total Runtime:", time.time() - start_ts)

# saving the model
#model1.save('Xception_phase1-1.h5')

In [ ]:
# accuracy vs. epoch and loss vs. epoch graphs
#plt.figure(figsize = (10, 5))
#plt.subplot(1, 2, 1)
#plt.plot(history1.history['acc']), plt.plot(history1.history['val_acc'])
#plt.title('Model Accuracy'), plt.ylabel('Accuracy'), plt.xlabel('Epoch')
#plt.legend(['train', 'test'], loc = 'upper left')
#plt.subplot(1, 2, 2)
#plt.plot(history1.history['loss']), plt.plot(history1.history['val_loss'])
#plt.title('Model Loss'), plt.ylabel('Loss'), plt.xlabel('Epoch')
#plt.legend(['train', 'test'], loc = 'upper left')
#plt.show()

In [ ]:
# at this point, the top layers are well trained and we can start fine-tuning convolutional layers 
# from inception V3. We will freeze the bottom N layers and train the remaining top layers.
# let's visualize layer names and layer indices to see how many layers we should freeze:
#for i, layer in enumerate(model1.layers):
#    print(i, layer.name)

In [ ]:
# loading the model
model1 = keras.models.load_model('Xception_phase1-2.h5')

In [ ]:
# we chose to train the top 2 inception blocks, i.e. we will freeze the first 249 layers and unfreeze the rest:
for layer in model1.layers[:115]:
    print(layer.trainable)# = False
for layer in model1.layers[115:]:
    print(layer.trainable)# = True

In [ ]:
# learning rate finder
#lr_finder1_1 = ConvNet_sungil.lr_finder(model1)
#lr_finder1_1.find_generator(train_datagen.flow(und_x_train, y_train_bin, batch_size = 256),\
#                          start_lr = 0.00001, end_lr = 0.1, epochs = 2, steps_per_epoch = len(und_x_train) / 256)

In [ ]:
#lr_finder1_1.plot_loss(n_skip_beginning = 1)
#lr_finder1_1.plot_loss_change(n_skip_beginning = 1, sma = 20)

In [ ]:
# finetuning and recompiling the model for unfreezing to take effect
#model1.compile(optimizer = keras.optimizers.Adam(lr = 2*(10**(-5))), loss = 'categorical_crossentropy',\
#               metrics = ['accuracy'])

# patient early stopping and model checkpoint
es = keras.callbacks.EarlyStopping(monitor = 'val_loss', mode = 'min', verbose = 1, patience = 5)
mc = keras.callbacks.ModelCheckpoint('Xception_phase1-2.h5', monitor = 'val_acc', mode = 'max',\
                                     verbose = 1, save_best_only = True)

# model fitting
start_ts = time.time()
history1_1 = model1.fit_generator(train_datagen.flow(und_x_train, y_train_bin, batch_size = 256),\
                epochs = 10, steps_per_epoch = len(und_x_train) / 256, callbacks = [es, mc],\
                validation_data = test_datagen.flow(x_test, y_test_bin),\
                validation_steps = len(x_test) / 256)
print("Total Runtime:", time.time()-start_ts)

In [ ]:
# accuracy vs. epoch and loss vs. epoch graphs
plt.figure(figsize = (10, 5))
plt.subplot(1, 2, 1)
plt.plot(history1_1.history['acc']), plt.plot(history1_1.history['val_acc'])
plt.title('Model Accuracy'), plt.ylabel('Accuracy'), plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc = 'upper left')
plt.subplot(1, 2, 2)
plt.plot(history1_1.history['loss']), plt.plot(history1_1.history['val_loss'])
plt.title('Model Loss'), plt.ylabel('Loss'), plt.xlabel('Epoch')
plt.legend(['train', 'test'], loc = 'upper left')
plt.show()

In [ ]:
# saving the model
model1.save('Xception_phase1-2.h5')